In [44]:
import pandas as pd
from sqlalchemy import create_engine

In [76]:
csv_file = "../ETL_PROJECT/Datasets/MoviesOnStreamingPlatforms_updated 2.csv"
online_data_df = pd.read_csv(csv_file)
online_data_df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [77]:
new_online_data_df = online_data_df[['ID','Title','Year','Age','IMDb','Rotten Tomatoes','Netflix','Hulu','Prime Video', 
                                         'Disney+','Type', 'Directors','Genres', 'Country','Language','Runtime']].copy()                                 
new_online_data_df

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16739,16740,The Ghosts of Buxley Hall,1980,NaN,6.2,NaN,0,0,0,1,0,Bruce Bilson,"Comedy,Family,Fantasy,Horror",United States,English,120.0
16740,16741,The Poof Point,2001,7+,4.7,NaN,0,0,0,1,0,Neal Israel,"Comedy,Family,Sci-Fi",United States,English,90.0
16741,16742,Sharks of Lost Island,2013,NaN,5.7,NaN,0,0,0,1,0,Neil Gelinas,Documentary,United States,English,NaN
16742,16743,Man Among Cheetahs,2017,NaN,6.6,NaN,0,0,0,1,0,Richard Slater-Jones,Documentary,United States,English,NaN


In [83]:
# rename columns Rotten Tomatoes and Prime Video names with underscores to import dataset into postgres tables
new_online_data_df.columns = ['id','title','year', 'age', 'imdb_score', 'rotten_tomatoes','netflix','hulu','prime_video', 
                                         'disney','type', 'directors','genres', 'country','language','runtime']
new_online_data_df.head()

,id,title,year,age,imdb_score,rotten_tomatoes,netflix,hulu,prime_video,disney,type,directors,genres,country,language,runtime
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [84]:
# Replacing infinite with nan the dropping all the rows with nan values in the given Pandas DataFrame
import numpy as np
new_online_data_df.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# Dropping all the rows with nan values
new_online_data_df.dropna(inplace=True)
  
# Printing df
new_online_data_df

,id,title,year,age,imdb_score,rotten_tomatoes,netflix,hulu,prime_video,disney,type,directors,genres,country,language,runtime
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16657,16658,A Kid in King Arthur's Court,1995,7+,4.7,5%,0,0,0,1,0,Michael Gottlieb,"Adventure,Comedy,Family,Fantasy,Romance","United States,Hungary,United Kingdom",English,89.0
16671,16672,George of the Jungle 2,2003,7+,3.3,17%,0,0,0,1,0,David Grossman,"Adventure,Comedy,Family","United States,Australia",English,87.0
16677,16678,That Darn Cat,1997,7+,4.7,13%,0,0,0,1,0,Robert Stevenson,"Comedy,Crime,Family,Thriller",United States,"English,French",116.0
16687,16688,Kazaam,1996,7+,3.0,6%,0,0,0,1,0,Paul Michael Glaser,"Comedy,Family,Fantasy,Musical",United States,English,93.0


In [86]:
# Remove % from Rotten Tomatoes column

new_online_data_df['rotten_tomatoes'] = new_online_data_df.rotten_tomatoes.str.replace('%', ' ')

new_online_data_df

,id,title,year,age,imdb_score,rotten_tomatoes,netflix,hulu,prime_video,disney,type,directors,genres,country,language,runtime
0,1,Inception,2010,13+,8.8,87,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,87,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,3,Avengers: Infinity War,2018,13+,8.5,84,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,4,Back to the Future,1985,7+,8.5,96,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16657,16658,A Kid in King Arthur's Court,1995,7+,4.7,5,0,0,0,1,0,Michael Gottlieb,"Adventure,Comedy,Family,Fantasy,Romance","United States,Hungary,United Kingdom",English,89.0
16671,16672,George of the Jungle 2,2003,7+,3.3,17,0,0,0,1,0,David Grossman,"Adventure,Comedy,Family","United States,Australia",English,87.0
16677,16678,That Darn Cat,1997,7+,4.7,13,0,0,0,1,0,Robert Stevenson,"Comedy,Crime,Family,Thriller",United States,"English,French",116.0
16687,16688,Kazaam,1996,7+,3.0,6,0,0,0,1,0,Paul Michael Glaser,"Comedy,Family,Fantasy,Musical",United States,English,93.0


In [87]:
csv_file = "../ETL_PROJECT/Datasets/36238_IMDd-Movie-Database.csv"
theater_data_df = pd.read_csv(csv_file)
theater_data_df.head()

,Title,Release Date,Color/B&W,Genre,Language,Country,Rating,Lead Actor,Director Name,Lead Actor FB Likes,Cast FB Likes,Director FB Likes,Movie FB Likes,IMDb Score (1-10),Total Reviews,Duration (min),Gross Revenue,Budget
0,Over the Hill to the Poorhouse,9/15/1920,Black and White,Crime,English,USA,Not Rated,Stephen Carr,Harry F. Millarde,2.0,4,0,0,4.8,1.0,110.0,3000000,100000
1,Metropolis,1/26/1927,Black and White,Drama,German,Germany,Not Rated,Brigitte Helm,Fritz Lang,136.0,203,756,12000,8.3,260.0,145.0,26435,6000000
2,The Broadway Melody,11/11/1929,Black and White,Musical,English,USA,Passed,Anita Page,Harry Beaumont,77.0,109,4,167,6.3,36.0,100.0,2808000,379000
3,42nd Street,8/29/1933,Black and White,Comedy,English,USA,Unrated,Ginger Rogers,Lloyd Bacon,610.0,995,24,439,7.7,65.0,89.0,2300000,439000
4,Top Hat,4/15/1935,Black and White,Comedy,English,USA,Approved,Ginger Rogers,Mark Sandrich,610.0,824,10,1000,7.8,66.0,81.0,3000000,609000


In [88]:
csv_file = "../ETL_PROJECT/Datasets/36238_IMDd-Movie-Database.csv"
theater_data_df = pd.read_csv(csv_file)
theater_data_df.head()

,Title,Release Date,Color/B&W,Genre,Language,Country,Rating,Lead Actor,Director Name,Lead Actor FB Likes,Cast FB Likes,Director FB Likes,Movie FB Likes,IMDb Score (1-10),Total Reviews,Duration (min),Gross Revenue,Budget
0,Over the Hill to the Poorhouse,9/15/1920,Black and White,Crime,English,USA,Not Rated,Stephen Carr,Harry F. Millarde,2.0,4,0,0,4.8,1.0,110.0,3000000,100000
1,Metropolis,1/26/1927,Black and White,Drama,German,Germany,Not Rated,Brigitte Helm,Fritz Lang,136.0,203,756,12000,8.3,260.0,145.0,26435,6000000
2,The Broadway Melody,11/11/1929,Black and White,Musical,English,USA,Passed,Anita Page,Harry Beaumont,77.0,109,4,167,6.3,36.0,100.0,2808000,379000
3,42nd Street,8/29/1933,Black and White,Comedy,English,USA,Unrated,Ginger Rogers,Lloyd Bacon,610.0,995,24,439,7.7,65.0,89.0,2300000,439000
4,Top Hat,4/15/1935,Black and White,Comedy,English,USA,Approved,Ginger Rogers,Mark Sandrich,610.0,824,10,1000,7.8,66.0,81.0,3000000,609000


In [52]:
# convert the Release Date to datetime
theater_data_df['Release Date'] = pd.to_datetime(theater_data_df['Release Date'])
# add a column for Year
theater_data_df['Year'] = theater_data_df['Release Date'].dt.year
# print the dataframe
theater_data_df

,Title,Release Date,Color/B&W,Genre,Language,Country,Rating,Lead Actor,Director Name,Lead Actor FB Likes,Cast FB Likes,Director FB Likes,Movie FB Likes,IMDb Score (1-10),Total Reviews,Duration (min),Gross Revenue,Budget,Year
0,Over the Hill to the Poorhouse,1920-09-15,Black and White,Crime,English,USA,Not Rated,Stephen Carr,Harry F. Millarde,2.0,4,0,0,4.8,1.0,110.0,3000000,100000,1920
1,Metropolis,1927-01-26,Black and White,Drama,German,Germany,Not Rated,Brigitte Helm,Fritz Lang,136.0,203,756,12000,8.3,260.0,145.0,26435,6000000,1927
2,The Broadway Melody,1929-11-11,Black and White,Musical,English,USA,Passed,Anita Page,Harry Beaumont,77.0,109,4,167,6.3,36.0,100.0,2808000,379000,1929
3,42nd Street,1933-08-29,Black and White,Comedy,English,USA,Unrated,Ginger Rogers,Lloyd Bacon,610.0,995,24,439,7.7,65.0,89.0,2300000,439000,1933
4,Top Hat,1935-04-15,Black and White,Comedy,English,USA,Approved,Ginger Rogers,Mark Sandrich,610.0,824,10,1000,7.8,66.0,81.0,3000000,609000,1935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3720,Unfinished Business,2015-02-25,Color,Comedy,English,USA,R,Tom Wilkinson,Ken Scott,1000.0,1564,31,0,5.4,90.0,91.0,10214013,35000000,2015
3721,Vacation,2015-08-03,Color,Adventure,English,USA,R,Chris Hemsworth,John Francis Daley,26000.0,40312,0,28000,6.1,204.0,99.0,58879132,31000000,2015
3722,Victor Frankenstein,2015-03-08,Color,Drama,English,USA,PG-13,Daniel Radcliffe,Paul McGuigan,11000.0,12876,118,11000,6.0,159.0,110.0,5773519,40000000,2015
3723,We Are Your Friends,2015-05-14,Color,Drama,English,UK,R,Vanessa Lengies,Max Joseph,804.0,3013,31,0,6.1,158.0,96.0,3590010,2000000,2015


In [89]:
# remove Release Date columns and re-arrange Year to the first columm
new_theater_data_df = theater_data_df[['Title','Release Date','Year','Color/B&W', 'Genre', 'Language', 'Country','Rating','Lead Actor',
                                         'Director Name','Lead Actor FB Likes', 'Cast FB Likes','Director FB Likes', 
                                         'Movie FB Likes','IMDb Score (1-10)','Total Reviews',
                                         'Duration (min)', 'Gross Revenue','Budget']].copy()                   
new_theater_data_df.head()

KeyError: "['Year'] not in index"

In [10]:
# rename the following columns names below with underscores to import dataset into postgres tables
#Lead Actor,Director Name,Lead Actor FB Likes,Cast FB Likes,Director FB Likes,Movie FB Likes,IMDb Score (1-10)
#Total Reviews,Duration (min),Gross Revenue'
new_theater_data_df.columns = ['year','title','release_date','color_bw','genre','language','country','rating','lead_actor','director_name','lead_actor_fb_likes'
                               ,'cast_fb_likes','director_fb_likes','movie_fb_likes','imdb_score','total_reviews',
                              'duration_mins','gross_revenue','budget']

new_theater_data_df

,year,title,release_date,color_bw,genre,language,country,rating,lead_actor,director_name,lead_actor_fb_likes,cast_fb_likes,director_fb_likes,movie_fb_likes,imdb_score,total_reviews,duration_mins,gross_revenue,budget
0,Over the Hill to the Poorhouse,1920-09-15,1920,Black and White,Crime,English,USA,Not Rated,Stephen Carr,Harry F. Millarde,2.0,4,0,0,4.8,1.0,110.0,3000000,100000
1,Metropolis,1927-01-26,1927,Black and White,Drama,German,Germany,Not Rated,Brigitte Helm,Fritz Lang,136.0,203,756,12000,8.3,260.0,145.0,26435,6000000
2,The Broadway Melody,1929-11-11,1929,Black and White,Musical,English,USA,Passed,Anita Page,Harry Beaumont,77.0,109,4,167,6.3,36.0,100.0,2808000,379000
3,42nd Street,1933-08-29,1933,Black and White,Comedy,English,USA,Unrated,Ginger Rogers,Lloyd Bacon,610.0,995,24,439,7.7,65.0,89.0,2300000,439000
4,Top Hat,1935-04-15,1935,Black and White,Comedy,English,USA,Approved,Ginger Rogers,Mark Sandrich,610.0,824,10,1000,7.8,66.0,81.0,3000000,609000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3720,Unfinished Business,2015-02-25,2015,Color,Comedy,English,USA,R,Tom Wilkinson,Ken Scott,1000.0,1564,31,0,5.4,90.0,91.0,10214013,35000000
3721,Vacation,2015-08-03,2015,Color,Adventure,English,USA,R,Chris Hemsworth,John Francis Daley,26000.0,40312,0,28000,6.1,204.0,99.0,58879132,31000000
3722,Victor Frankenstein,2015-03-08,2015,Color,Drama,English,USA,PG-13,Daniel Radcliffe,Paul McGuigan,11000.0,12876,118,11000,6.0,159.0,110.0,5773519,40000000
3723,We Are Your Friends,2015-05-14,2015,Color,Drama,English,UK,R,Vanessa Lengies,Max Joseph,804.0,3013,31,0,6.1,158.0,96.0,3590010,2000000


In [90]:
# Replacing infinite with nan the dropping all the rows with nan values in the given Pandas DataFrame
import numpy as np
new_theater_data_df.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# Dropping all the rows with nan values
new_theater_data_df.dropna(inplace=True)
  
# Printing df
new_theater_data_df

,Title,Release Date,Year,Color/B&W,Genre,Language,Country,Rating,Lead Actor,Director Name,Lead Actor FB Likes,Cast FB Likes,Director FB Likes,Movie FB Likes,IMDb Score (1-10),Total Reviews,Duration (min),Gross Revenue,Budget
0,Over the Hill to the Poorhouse,1920-09-15,1920,Black and White,Crime,English,USA,Not Rated,Stephen Carr,Harry F. Millarde,2.0,4,0,0,4.8,1.0,110.0,3000000,100000
1,Metropolis,1927-01-26,1927,Black and White,Drama,German,Germany,Not Rated,Brigitte Helm,Fritz Lang,136.0,203,756,12000,8.3,260.0,145.0,26435,6000000
2,The Broadway Melody,1929-11-11,1929,Black and White,Musical,English,USA,Passed,Anita Page,Harry Beaumont,77.0,109,4,167,6.3,36.0,100.0,2808000,379000
3,42nd Street,1933-08-29,1933,Black and White,Comedy,English,USA,Unrated,Ginger Rogers,Lloyd Bacon,610.0,995,24,439,7.7,65.0,89.0,2300000,439000
4,Top Hat,1935-04-15,1935,Black and White,Comedy,English,USA,Approved,Ginger Rogers,Mark Sandrich,610.0,824,10,1000,7.8,66.0,81.0,3000000,609000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3720,Unfinished Business,2015-02-25,2015,Color,Comedy,English,USA,R,Tom Wilkinson,Ken Scott,1000.0,1564,31,0,5.4,90.0,91.0,10214013,35000000
3721,Vacation,2015-08-03,2015,Color,Adventure,English,USA,R,Chris Hemsworth,John Francis Daley,26000.0,40312,0,28000,6.1,204.0,99.0,58879132,31000000
3722,Victor Frankenstein,2015-03-08,2015,Color,Drama,English,USA,PG-13,Daniel Radcliffe,Paul McGuigan,11000.0,12876,118,11000,6.0,159.0,110.0,5773519,40000000
3723,We Are Your Friends,2015-05-14,2015,Color,Drama,English,UK,R,Vanessa Lengies,Max Joseph,804.0,3013,31,0,6.1,158.0,96.0,3590010,2000000


In [12]:
# online streaming dataset
new_online_data_df
new_online_data_df.head()
new_online_data_df.drop_duplicates()

,id,title,year,age,imdb_score,rotten_tomatoes,netflix,hulu,prime_video,disney,type,directors,genres,country,language,runtime
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16657,16658,A Kid in King Arthur's Court,1995,7+,4.7,5%,0,0,0,1,0,Michael Gottlieb,"Adventure,Comedy,Family,Fantasy,Romance","United States,Hungary,United Kingdom",English,89.0
16671,16672,George of the Jungle 2,2003,7+,3.3,17%,0,0,0,1,0,David Grossman,"Adventure,Comedy,Family","United States,Australia",English,87.0
16677,16678,That Darn Cat,1997,7+,4.7,13%,0,0,0,1,0,Robert Stevenson,"Comedy,Crime,Family,Thriller",United States,"English,French",116.0
16687,16688,Kazaam,1996,7+,3.0,6%,0,0,0,1,0,Paul Michael Glaser,"Comedy,Family,Fantasy,Musical",United States,English,93.0


In [55]:
# theater dataset
new_theater_data_df
new_theater_data_df.head()
new_theater_data_df.drop_duplicates()

,Title,Release Date,Year,Color/B&W,Genre,Language,Country,Rating,Lead Actor,Director Name,Lead Actor FB Likes,Cast FB Likes,Director FB Likes,Movie FB Likes,IMDb Score (1-10),Total Reviews,Duration (min),Gross Revenue,Budget
0,Over the Hill to the Poorhouse,1920-09-15,1920,Black and White,Crime,English,USA,Not Rated,Stephen Carr,Harry F. Millarde,2.0,4,0,0,4.8,1.0,110.0,3000000,100000
1,Metropolis,1927-01-26,1927,Black and White,Drama,German,Germany,Not Rated,Brigitte Helm,Fritz Lang,136.0,203,756,12000,8.3,260.0,145.0,26435,6000000
2,The Broadway Melody,1929-11-11,1929,Black and White,Musical,English,USA,Passed,Anita Page,Harry Beaumont,77.0,109,4,167,6.3,36.0,100.0,2808000,379000
3,42nd Street,1933-08-29,1933,Black and White,Comedy,English,USA,Unrated,Ginger Rogers,Lloyd Bacon,610.0,995,24,439,7.7,65.0,89.0,2300000,439000
4,Top Hat,1935-04-15,1935,Black and White,Comedy,English,USA,Approved,Ginger Rogers,Mark Sandrich,610.0,824,10,1000,7.8,66.0,81.0,3000000,609000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3720,Unfinished Business,2015-02-25,2015,Color,Comedy,English,USA,R,Tom Wilkinson,Ken Scott,1000.0,1564,31,0,5.4,90.0,91.0,10214013,35000000
3721,Vacation,2015-08-03,2015,Color,Adventure,English,USA,R,Chris Hemsworth,John Francis Daley,26000.0,40312,0,28000,6.1,204.0,99.0,58879132,31000000
3722,Victor Frankenstein,2015-03-08,2015,Color,Drama,English,USA,PG-13,Daniel Radcliffe,Paul McGuigan,11000.0,12876,118,11000,6.0,159.0,110.0,5773519,40000000
3723,We Are Your Friends,2015-05-14,2015,Color,Drama,English,UK,R,Vanessa Lengies,Max Joseph,804.0,3013,31,0,6.1,158.0,96.0,3590010,2000000


In [91]:
# Connect to local postgres database
from sqlalchemy import create_engine
rds_connection_string = "postgres:changeit@localhost:5432/movies"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [92]:
# Check for tables
engine.table_names()

['online', 'theater']

In [93]:
new_online_data_df

,id,title,year,age,imdb_score,rotten_tomatoes,netflix,hulu,prime_video,disney,type,directors,genres,country,language,runtime
0,1,Inception,2010,13+,8.8,87,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,87,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,3,Avengers: Infinity War,2018,13+,8.5,84,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,4,Back to the Future,1985,7+,8.5,96,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16657,16658,A Kid in King Arthur's Court,1995,7+,4.7,5,0,0,0,1,0,Michael Gottlieb,"Adventure,Comedy,Family,Fantasy,Romance","United States,Hungary,United Kingdom",English,89.0
16671,16672,George of the Jungle 2,2003,7+,3.3,17,0,0,0,1,0,David Grossman,"Adventure,Comedy,Family","United States,Australia",English,87.0
16677,16678,That Darn Cat,1997,7+,4.7,13,0,0,0,1,0,Robert Stevenson,"Comedy,Crime,Family,Thriller",United States,"English,French",116.0
16687,16688,Kazaam,1996,7+,3.0,6,0,0,0,1,0,Paul Michael Glaser,"Comedy,Family,Fantasy,Musical",United States,English,93.0


In [94]:
# Use pandas to load online csv converted DataFrame into database
new_online_data_df.to_sql(name='online', con=engine, if_exists='append', index=False)

In [21]:
new_theater_data_df

,year,title,release_date,color_bw,genre,language,country,rating,lead_actor,director_name,lead_actor_fb_likes,cast_fb_likes,director_fb_likes,movie_fb_likes,imdb_score,total_reviews,duration_mins,gross_revenue,budget
0,Over the Hill to the Poorhouse,1920-09-15,1920,Black and White,Crime,English,USA,Not Rated,Stephen Carr,Harry F. Millarde,2.0,4,0,0,4.8,1.0,110.0,3000000,100000
1,Metropolis,1927-01-26,1927,Black and White,Drama,German,Germany,Not Rated,Brigitte Helm,Fritz Lang,136.0,203,756,12000,8.3,260.0,145.0,26435,6000000
2,The Broadway Melody,1929-11-11,1929,Black and White,Musical,English,USA,Passed,Anita Page,Harry Beaumont,77.0,109,4,167,6.3,36.0,100.0,2808000,379000
3,42nd Street,1933-08-29,1933,Black and White,Comedy,English,USA,Unrated,Ginger Rogers,Lloyd Bacon,610.0,995,24,439,7.7,65.0,89.0,2300000,439000
4,Top Hat,1935-04-15,1935,Black and White,Comedy,English,USA,Approved,Ginger Rogers,Mark Sandrich,610.0,824,10,1000,7.8,66.0,81.0,3000000,609000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3720,Unfinished Business,2015-02-25,2015,Color,Comedy,English,USA,R,Tom Wilkinson,Ken Scott,1000.0,1564,31,0,5.4,90.0,91.0,10214013,35000000
3721,Vacation,2015-08-03,2015,Color,Adventure,English,USA,R,Chris Hemsworth,John Francis Daley,26000.0,40312,0,28000,6.1,204.0,99.0,58879132,31000000
3722,Victor Frankenstein,2015-03-08,2015,Color,Drama,English,USA,PG-13,Daniel Radcliffe,Paul McGuigan,11000.0,12876,118,11000,6.0,159.0,110.0,5773519,40000000
3723,We Are Your Friends,2015-05-14,2015,Color,Drama,English,UK,R,Vanessa Lengies,Max Joseph,804.0,3013,31,0,6.1,158.0,96.0,3590010,2000000


In [23]:
# Use pandas to load theater csv converted DataFrame into database
new_theater_data_df.to_sql(name='theater', con=engine, if_exists='append', index=False)

In [24]:
pd.read_sql_query('select * from online', con=engine).head()

,id,title,year,age,imdb_score,rotten_tomatoes,netflix,hulu,prime_video,disney,type,directors,genres,country,language,runtime
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161


In [25]:
pd.read_sql_query('select * from theater', con=engine).head()

,year,title,release_date,color_bw,genre,language,country,rating,lead_actor,director_name,lead_actor_fb_likes,cast_fb_likes,director_fb_likes,movie_fb_likes,imdb_score,total_reviews,duration_mins,gross_revenue,budget
0,Over the Hill to the Poorhouse,1920-09-15 00:00:00,1920,Black and White,Crime,English,USA,Not Rated,Stephen Carr,Harry F. Millarde,2,4,0,0,4.8,1,110,3000000,100000
1,Metropolis,1927-01-26 00:00:00,1927,Black and White,Drama,German,Germany,Not Rated,Brigitte Helm,Fritz Lang,136,203,756,12000,8.3,260,145,26435,6000000
2,The Broadway Melody,1929-11-11 00:00:00,1929,Black and White,Musical,English,USA,Passed,Anita Page,Harry Beaumont,77,109,4,167,6.3,36,100,2808000,379000
3,42nd Street,1933-08-29 00:00:00,1933,Black and White,Comedy,English,USA,Unrated,Ginger Rogers,Lloyd Bacon,610,995,24,439,7.7,65,89,2300000,439000
4,Top Hat,1935-04-15 00:00:00,1935,Black and White,Comedy,English,USA,Approved,Ginger Rogers,Mark Sandrich,610,824,10,1000,7.8,66,81,3000000,609000


In [26]:
pd.read_sql_query('select a.*,b.* \
          from online a\
          left join theater b\
          on a.year=b.year\
          and a.title=b.title', con=engine).head()

,id,title,year,age,imdb_score,rotten_tomatoes,netflix,hulu,prime_video,disney,...,director_name,lead_actor_fb_likes,cast_fb_likes,director_fb_likes,movie_fb_likes,imdb_score,total_reviews,duration_mins,gross_revenue,budget
0,16512,Return from Witch Mountain,1978,all,5.9,50%,0,0,0,1,...,None,None,None,None,None,None,None,None,None,None
1,503,Soul Surfer,2011,7+,7.0,45%,1,0,0,0,...,None,None,None,None,None,None,None,None,None,None
2,8808,Jack and Jill vs. the World,2008,13+,5.5,25%,0,0,1,0,...,None,None,None,None,None,None,None,None,None,None
3,5030,This Beautiful Fantastic,2016,7+,6.9,61%,0,0,1,0,...,None,None,None,None,None,None,None,None,None,None
4,6739,Bruno,2000,13+,6.1,20%,0,0,1,0,...,None,None,None,None,None,None,None,None,None,None
